In [12]:
def parse_input(file):
    with open(file) as file_in:
        wires, gates = file_in.read().split('\n\n')

    wire_dict = {}
    for row in wires.splitlines():
        wire, value = row.split(': ')
        wire_dict[wire] = int(value)
    
    op_name2sign = {'AND': '&', 'OR': '|', 'XOR': '^'}
    gates_parsed = []
    for row in gates.splitlines():
        rule, out = row.split(' -> ')
        in1, op, in2 = rule.split(' ')
        gates_parsed.append((in1, op_name2sign[op], in2, out))

    return wire_dict, gates_parsed

In [14]:
wires, gates = parse_input('example1.txt')